In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import geocoder

ImportError: No module named 'geocoder'

In [3]:
#getting the sourse html of the webpage
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#creating the BS object to work with
soup = BeautifulSoup(source, 'lxml')

In [4]:
#fetching the table component from the html wikipedia page
table = soup.find("table") 

#each cell represents a row in the future DataFrame
cells = table.find_all("tr")

#we will make DataFrame from these arrays
PostalCode   = list()
Borough      = list()
Neighborhood = list()

#going through the cells and filling out the arrays
for i in range(1,len(cells)):
    current_elems = cells[i].find_all("td")
    
    nbhood = str(current_elems[2].text) #some issues with the last column 
    
    PostalCode.append(current_elems[0].text)
    Borough.append(current_elems[1].text)
    Neighborhood.append(nbhood[0:len(nbhood)-1])

In [5]:
data_ = {"PostalCode" : PostalCode, "Borough": Borough, "Neighborhood": Neighborhood}
df =pd.DataFrame(data=data_)
df.head()

,Borough,Neighborhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [6]:
#Processing the DataFrame according to given conditions

#1: all entries with Borough = Not Assigned must be dropped

df = df.loc[df['Borough'] != 'Not assigned'].reset_index()


#2: If neighborhood is unavailable, we copy borough into it

df.loc[df['Neighborhood'] == "Not assigned", 'Neighborhood'] = df['Borough']

df = df[['PostalCode','Borough','Neighborhood']]

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
#3: grouping neighborhoods by postal code

for elem in df['PostalCode'].tolist():
    #print('elem: ' + str(elem))
    borough = df.loc[df['PostalCode'] == elem,'Borough'].tolist()[0]
    neighborhoods = df.loc[df['PostalCode']==elem,'Neighborhood'].tolist()
    neighbString  = ""
    
    for i in neighborhoods:
        neighbString += (i+", ")
        
    df = df.loc[df['PostalCode'] != elem]
    df = df.append({'PostalCode': elem, 'Borough': borough, 'Neighborhood': neighbString[0:-2]},ignore_index=True)
    df.reset_index()

In [8]:
df.shape

(103, 3)

In [9]:
llframe = pd.read_csv("http://cocl.us/Geospatial_data")
llframe.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
postalcodes = df['PostalCode'].tolist()
latitudes   = list()
longitudes  = list()

for code in postalcodes:
        lat = llframe.loc[llframe['Postal Code'] == code, 'Latitude'].tolist()[0]
        lng = llframe.loc[llframe['Postal Code'] == code, 'Longitude'].tolist()[0]
        latitudes.append(lat)
        longitudes.append(lng)
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude']= pd.Series(longitudes,index=df.index)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:

df = df.loc[df['Borough'].str.contains("Toronto")].reset_index()
df.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [12]:
df.shape

(38, 6)

In [13]:
import folium
from geopy.geocoders import Nominatim

ImportError: No module named 'folium'

In [14]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

NameError: name 'Nominatim' is not defined

In [15]:
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=11)

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)
    
map_toronto

NameError: name 'folium' is not defined